# Optimized Meal Planning

## Define the model

Data

$$
\begin{aligned}
T, \text{the set of discrete time periods} \\
I, \text{the set of ingredients that are available} \\
N, \text{the set of nutrients that are tracked} \\
R, \text{the set of known recipes} \\
F_{ij}, \text{the amount of nutrient } j \text{ in a unit amount of ingredient } i,\ \forall i \in I,\ j \in N \\
S_{j}, \text{the amount of nutrient } j \text{ that is needed each week for a healthy diet} \\
C_{i}, \text{the unit cost of ingredient } i,\ \forall i \in I \\
V_{i}, \text{the unit volume of ingredient } i,\ \forall i \in I \\
Fr, \text{the max volume of the fridge} \\
Req_{il}, \text{the amount of ingredient i needed for recipe} \ l \ \forall i \in I, l \in R \\
MAX, \text{the maximum number of recipes that can be made in a week} (tentative)
\end{aligned}
$$


Decision variables:
- $x_{ik}$, the amount of ingredient i bought in week k, $\forall i \in I, k \in T$
- $y_{lk}$, the amount of recipe $l$ to make in week k, $\forall l \in R, k \in T$
- $z_{ik}$, the amount of ingredient i to store in the fridge in week k, $\forall i \in I, k \in T$

Linear Program:
\begin{align}
\min \sum_{k \in T} \sum_{i \in I} x_{ik} * C_i \\
\text{s.t. } \sum_{l \in R} \sum_{i \in I} y_{lk} * Req_{il} * F_{ij} \geq S_{j}, \forall k \in T, j \in N \\
\sum_{i \in I} z_{ik} * V_{i} \leq Fr, \forall k \in T \\
z_{i0} = 0, \forall i \in I \\
x_{ik} + z_{i,k-1} = \sum_{l \in R} y_{lk} * Req_{il} + z_{ik}, \forall k \in T, i \in I \\
\sum_{l \in R} y_{lk} \leq MAX, \forall k \in T \\
y_{lk}, x_{ik}, z_{ik} \geq 0, \forall i \in I, k \in T
\end{align}

- The first constraint ensures there are enough nutrients
- The second constraint ensures that the fridge is not overfilled
- The third constraint sets the intial amount of food to 0
- The fourth constraint ensures that the amount of each ingredient being consumed is balanced by the amount being bought
- The fifth constraint limits the amount of recipes made in a week (i.e. only 7 recipes worth of recipes being made)

In [39]:
# for now leave out multi-period planning and general form - too complex

I = ["rice", "chicken nuggets", "steak"]
N = ["calories", "protein"]
R = ["steak and rice", "just nuggets"]
F = [300 0; 200 5; 400 30]
S = [2000, 100]
C = [5, 5, 15]
Req = [1 0; 0 1; 1 0]

# make the model
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)

@variable(m, x[1:3] >= 0)
@variable(m, y[1:2] >= 0)

@constraint(m, nutr[j in 1:2], sum(y[l] * sum(Req[i,l] * F[i,j] for i in 1:3) for l in 1:2) >= S[j])
@constraint(m, init_balance[i in 1:3], x[i] >= sum( Req[i,l]*y[l] for l in 1:2 )) 

@objective(m, Min, sum( x[i] * C[i] for i in 1:3 ))

optimize!(m)

println("Buy ingredients ", value.(x))
println("Make recipes ", value.(y))
println("Total cost ", objective_value(m))

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 5 rows; 5 cols; 10 nonzeros
Coefficient ranges:
  Matrix [1e+00, 7e+02]
  Cost   [5e+00, 2e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+03]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Presolve : Reductions: rows 2(-3); columns 2(-3); elements 4(-6)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(900) 0s
          3     6.6666666667e+01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  6.6666666667e+01
P-D objective error :  1.0578800036e-16
HiGHS run time      :          0.00
Buy ingredients [3.3333333333333335, -0.0, 3.3333333333333335]
Make recipes [3.3333333333333335, 0.0]
Total cost 66.66666666666667


In [ ]:
#using DataFrames, CSV
#df = CSV.read("stigler.csv",DataFrame,delim=',')